In [29]:
import pandas as pd
import glob

In [30]:
import json

with open("intents.json", "r") as file:
    data = json.load(file)


In [31]:
# Hugging Face - Sentence Transformer


In [8]:
!pip install sentence-transformers

In [32]:
sentences = []
tags = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        sentences.append(pattern)
        tags.append(intent["tag"])





In [33]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



In [34]:
embeddings = model.encode(sentences, show_progress_bar=True)
embeddings = np.array(embeddings)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [35]:
print("Embeddings shape:", embeddings.shape)
# Should print something like (32, 384)


Embeddings shape: (33, 384)


In [12]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.8 MB/s eta 0:00:00


In [36]:
import faiss

dimensions = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimensions)
faiss_index.add(np.array(embeddings))



In [37]:
SIMILARITY_THRESHOLD = 0.5  # You can tune this


In [38]:
def chatbot_reply(user_input):
    if user_input.lower() in ["bye", "exit", "quit"]:
        return "Goodbye! Feel free to come back anytime.", True

    user_embedding = model.encode([user_input])
    D, I = faiss_index.search(np.array(user_embedding), k=1)

    best_distance = D[0][0]
    best_index = I[0][0]

    if best_distance > SIMILARITY_THRESHOLD:
        return ("I'm sorry, I can't provide information about that topic.\n"
                "You might find more details on these sites:\n"
                "- https://www.google.com\n"
                "- https://www.wikipedia.org\n"
                "- https://chat.openai.com"), False

    matched_tag = tags[best_index]
    for intent in data["intents"]:
        if intent["tag"] == matched_tag:
            return np.random.choice(intent["responses"]), False


In [39]:
while True:
    user_input = input("You: ")
    response, should_exit = chatbot_reply(user_input)
    print("Bot:", response)
    if should_exit:
        break


You: hye
Bot: Hi there! Looking for university info? I'm here to help.
You: university information
Bot: Need help with admissions, departments, or tuition? Just let me know!
You: courses
Bot: Explore a wide range of degree programs including BSc, BA, MBA, MTech, and more.
You: BA
Bot: I'm sorry, I can't provide information about that topic.
You might find more details on these sites:
- https://www.google.com
- https://www.wikipedia.org
- https://chat.openai.com
You: Departments
Bot: Each department is equipped with expert faculty and modern research facilities.
You: more
Bot: I'm sorry, I can't provide information about that topic.
You might find more details on these sites:
- https://www.google.com
- https://www.wikipedia.org
- https://chat.openai.com
You: bye
Bot: Goodbye! Feel free to come back anytime.


In [16]:
import json
from sentence_transformers import SentenceTransformer
import numpy as np

# Load intents
with open("intents.json", "r") as file:
    data = json.load(file)

# Extract patterns
sentences = []
tags = []
for intent in data["intents"]:
    for pattern in intent.get("patterns", []):
        sentences.append(pattern)
        tags.append(intent["tag"])

print(f"Extracted {len(sentences)} patterns.")

# Generate embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
embeddings = np.array(embeddings)

print("Embeddings shape:", embeddings.shape)  # Should now be (N, 384)


Extracted 46 patterns.
Embeddings shape: (46, 384)


In [ ]:

# Example interaction
user_input = "what courses are available?"
response = get_response(user_input)
print("Bot:", response)
